# Ayudantía Serialización de objetos
#### Gabriel Lyon y José Manuel Larraín

### Funciones útiles:

* dump(s)
* load(s)

#### Pickle
* \_\_getstate\_\_
* \_\_setstate\_\_

#### JSON
* JSONEncoder
* object_hook


## Introducción

¡Los alumnos del DDC tiene la idea del siglo! El servicio de mensajería instantánea DDChat, el que
permitirá comunicarse de forma confidencial entre los alumnos del DDC. Lamentablemente, el infame Dr. Mavrakis, logró acceder a la base de datos y leer todos los mensajes enviados por los alumnos.

Su misión es aplicar un sistema de seguridad encriptado para que no se vuelva a repetir tal desgracia,
pero como usted no se quiere quedar sin las últimas copuchas del DDC, implementará un sistema que le
permita visualizar las conversaciones entre dos alumnos.

### Formato Datos

#### Mensajes

Los mensajes vienen en el siguiente formato:

- {"send_to": 95521376, "content": "No puedo, tengo que estudiar", "send_by": 76251142, "last_view_date": "", "date": "18-4-2017 15:45"}

#### Usuarios

Los contactos vienen en el siguiente formato:

- {"name": "Francisca Rios", "contacts": [], "phone_number": 45348826}


### Requerimientos 

En primer lugar, deberá leer los archivos y transformar su información a objetos de las clases `Usuario` y
`Mensaje`. Esta información está contenida en la carpeta **db**, en donde la carpeta **usr** contiene a los usuarios y la carpeta **msg** contiene los mensajes. Ambos están en formato JSON.

Para lograr este paso deben crear dos funciones que retornen una lista conteniendo sus respectivos objetos.

In [1]:
import pickle
import json
import os
from random import randint
from datetime import datetime


class Usuario:
    def __init__(self, name, contacts, phone_number):
        self.name = name
        self.phone_number = phone_number
        self.contacts = contacts


class Mensaje:
    def __init__(self, send_to, content, send_by, date, last_view_date):
        self.send_to = send_to
        self.content = content
        self.send_by = send_by
        self.last_view_date = last_view_date
        self.date = date
        

    def __getstate__(self):
        message = self.__dict__.copy()
        message["content"] = caesarCipher(
            message["content"], message["send_by"])
        return message

    def __setstate__(self, state):
        d = datetime.now()
        state["last_view_date"] = "{0}/{1}/{2}-{3}:{4}".format(
            d.year, d.month, d.day, d.hour, d.minute)
        self.__dict__ = state
        
                
def readUsers():
    users = []
    for path in os.listdir("db/usr/"):
        with open("db/usr/" + path, "r") as f:
            user = json.load(f, object_hook=lambda dict_obj: Usuario(**dict_obj))
            users.append(user)
    return users


def readMessages():
    messages = []
    for path in os.listdir("db/msg"):
        with open("db/msg/" + path, "r") as f:
            message = json.load(f, object_hook=lambda dict_obj: Mensaje(**dict_obj))
        messages.append(message)
    return messages

Una vez cargado todos los datos, deberá agregar los contactos de cada usuario. Estos cuentan con el atributo `contacts`, que viene vacío. Para que un usuario **x** sea contacto de otro **y**, es necesario que **y** haya enviado un mensaje a **x**, pero **x** no necesariamente tiene a **y** como contacto.

In [2]:

# Aqui agregamos los numeros de telefono de sus contactos a los usuarios
# no se agrega a instancia por que para los prox
def updateUsers(users, messages):
    users = {user.phone_number: user for user in users}
    for message in messages:
        sender = message.send_by
        receptor = message.send_to
        if receptor not in users[sender].contacts:
            users[sender].contacts.append(receptor)


Luego deberá guardar los Usuarios actualizados en la carpeta **usr** de la carpeta **secure_db**, los cuales
deben quedar en formato JSON, con un archivo por usuario.

In [3]:

# Esta funcion es para serializar los usuarios
def saveUsers(users):
    for user in users:
        path = "secure_db/usr/{}".format(str(user.phone_number))
        with open(path, "w", encoding="utf-8") as data:
            json.dump(user.__dict__, data)


En el caso de los mensajes, que se guardan en la carpeta **msg** de **secure_db** utilizando pickle, deberá encriptar su contenido antes de guardarlos (un archivo por mensaje) con el algoritmo de seguridad más novedoso y seguro del momento: **Caesar Cipher**.

**Caesar Cipher** es muy simple; dado un string y un número n, cada uno de los caracteres se reemplazarán
por el n-ésimo caracter siguiente. Por ejemplo, dado n = 2 y la palabra Cesar, se obtendrá Eguct. El código
Cesar se resume en la siguiente fórmula: `y(x, n) = (x+n) mód 26`, donde cada letra **x** está asociada a un número y **n** son los números a desplazar **x**, e **y** es el nuevo valor del caracer. Una operación modular retorna el resto de la división. Por ejemplo (5 + 4) mód 2 = 1. Asuman que el alfabeto no incluye a la ñ ni caracteres acentuados.

¡La encriptación debe tomar lugar justo antes de serializar el archivo! La llave **n** a utilizar corresponderá
al número telefónico de quién envió el mensaje.

Finalmente, debe quedar registrado cada vez que alguien deserialice un mensaje. Para esto, deben actualizar en el archivo respectivo el atributo `last_view_date` con la fecha y hora de la última deserialización de un mensaje.

In [8]:

# Esta funcion sera llamada cada vez que se necesite encriptar un mensajes
# antes de ser serializado
def caesarCipher(string, key):
    new_string = ""
    for character in string:
        if character.isalpha():
            ascii_number = ord(character) - 97
            value = ascii_number + key
            value = value%26
            character = chr(value + 97)
        new_string += character
    return new_string



# Esta funcion es para encriptar los mensajes y serializarlos
def saveEncriptedMessages(messages):
    for message in messages:
        path = "secure_db/msg/{}".format(str(randint(0,999999)))
        with open(path, "wb") as data:
            pickle.dump(message, data)    


Finalmente ejecutamos el codigo llamando a las funciones.

In [10]:
if __name__ == '__main__':
    users = readUsers()
    messages = readMessages()
    updateUsers(users, messages)
    saveUsers(users)
    saveEncriptedMessages(messages)